# 长病程分析

In [1]:
import pandas as pd
import numpy as np
from data_analysis import data_tool as dt
import seaborn as sns
import matplotlib.pyplot as plt
# 确保系统中有中文字体，并设置为 SimHei
plt.rcParams['font.sans-serif'] = ['SimHei']  # 设置中文字体为黑体
plt.rcParams['axes.unicode_minus'] = False    # 解决坐标轴负号显示问题

In [37]:
df_date_start = pd.read_excel('dataset/发病情况.xlsx', sheet_name='首次发病及诊断情况')
df_ar = pd.read_excel('dataset/三个量表.xlsx',  sheet_name='ALSFRS-R评分量表')
df_date_start = df_date_start[['患者编号', '发病日期', '发病年龄', '确诊日期', '确诊年龄']]
df_ar = df_ar[['姓名', '患者编号', '评估日期', '总分']]
df=pd.read_excel('dataset/患者基本信息.xlsx')
df = df.replace(np.nan, None)

,姓名,性别,患者编号,出生日期,年龄,年龄(单位),民族,所在城市,紧急联系人手机号,家族史,患者成员数量,患病成员与您的关系,您的家族中是否有患肿瘤的成员,患病成员与您的关系.1,肿瘤类型,您的家族中是否有患以下疾病的成员
0,邝姘兰,女,202405120005,1965-12-01,58.0,岁,汉族,"广东省,珠海市",13160676640.0,无,None,None,有,配偶,肝癌,无
1,肖玲,女,202405120004,1974-11-01,49.0,岁,汉族,"广东省,东莞市",13712082979.0,无,None,None,无,None,None,None
2,叉叉,男,202405120003,1978-04-01,46.0,岁,汉族,"北京市,北京市",None,无,None,None,无,None,None,None
3,王利珍,男,202405120002,1959-01-01,65.0,岁,汉族,"河北省,张家口市",15831337661.0,无,None,None,无,None,None,None
4,赵青生,男,202405120001,1951-03-01,73.0,岁,汉族,"河北省,石家庄市",13011563015.0,无,None,None,无,None,None,None
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
13211,朱永森,男,202009170005,1976-07-01,46.0,岁,汉族,"河南省,三门峡市",15639850701.0,无,None,None,无,None,None,None
13212,管亚彬,男,202009170004,1974-09-25,47.0,岁,汉族,"湖北省,武汉市",18071426069.0,无,None,None,无,None,None,None
13213,武杰,男,202009170003,1978-12-01,43.0,岁,汉族,"内蒙古自治区,呼和浩特市",13394717892.0,无,None,None,无,None,None,None
13214,金冬浩,男,202009170002,1977-10-01,44.0,岁,汉族,"北京市,北京市",13811800712.0,无,None,None,无,None,None,None


# df_ar处理

In [32]:
df_ar = dt.df_merge_dict(df=df_ar, date_name='评估日期', test='总分')
df_ar = df_ar[['姓名','患者编号','总分提交记录']].drop_duplicates(subset='患者编号')
df_long_date = pd.merge(df_ar,df_date_start,on='患者编号',how='right')
df_long_date = df_long_date.dropna(subset='总分提交记录')

In [34]:
def extract_max_date_and_score(submission_dict):
    '''
    从提交记录字典中提取最大日期及其对应的分数
    :param submission_dict: 日期字典
    :return: (最大日期, 对应分数)
    '''
    # 确保 submission_dict 是字典
    if not isinstance(submission_dict, dict):
        return None, None

    if not submission_dict:
        return None, None

    # 提取字典的键（日期），并找到最大的日期
    max_date = max(submission_dict.keys())
    min_date = min(submission_dict.keys())

    # 提取最大日期对应的分数
    max_score = submission_dict[max_date][0] if submission_dict[max_date] else None
    min_score = submission_dict[min_date][0] if submission_dict[min_date] else None
    return max_date, max_score, min_date, min_score

# 应用函数
df_long_date[['最晚提交日期', '最晚分数','最早提交日期','最早分数']] = df_long_date['总分提交记录'].apply(lambda x: pd.Series(extract_max_date_and_score(x)))

df_long_date=df_long_date.dropna(subset='发病日期')


,姓名,患者编号,总分提交记录,发病日期,发病年龄,确诊日期,确诊年龄,最晚提交日期,最晚分数,最早提交日期,最早分数
3,朱建文,202310140001,{'2023-10-14': [39]},2022-09,66.0,2023-10-08,67.0,2023-10-14,39.0,2023-10-14,39.0
4,张戈,202310130019,{'2023-10-16': [29]},2022-10,68.0,2023-01-03,68.0,2023-10-16,29.0,2023-10-16,29.0
5,涂建新,202310130012,"{'2023-10-23': [39], '2023-12-12': [40]}",2021-04,50.0,2023-05-09,52.0,2023-12-12,40.0,2023-10-23,39.0
6,贾彩琴,202310130011,"{'2023-10-16': [33], '2023-11-13': [35], '2023...",2020-08,71.0,NaN,NaN,2024-01-11,38.0,2023-10-16,33.0
7,刘德全,202310120008,{'2024-01-11': [36]},2023-03,21.0,2023-07-25,22.0,2024-01-11,36.0,2024-01-11,36.0
...,...,...,...,...,...,...,...,...,...,...,...
6079,朱永森,202009170005,"{'2020-10-12': [36], '2020-12-07': [26]}",2018-12-15,42.0,2019-09-23,43.0,2020-12-07,26.0,2020-10-12,36.0
6080,管亚彬,202009170004,"{'2020-09-18': [36], '2020-11-10': [33], '2020...",2018-10-15,44.0,2019-06-15,44.0,2021-10-02,22.0,2020-09-18,36.0
6081,武杰,202009170003,"{'2020-11-10': [40], '2020-11-30': [39], '2020...",2016-10-15,37.0,2018-11-30,39.0,2022-11-20,20.0,2020-11-10,40.0
6082,金冬浩,202009170002,{'2020-11-20': [28]},2019-03-15,39.0,NaN,NaN,2020-11-20,28.0,2020-11-20,28.0


In [46]:
def dur(row):
    # 尝试将日期字符串转换为日期时间对象
    date1 = pd.to_datetime(row['发病日期'], errors='coerce')
    date2 = pd.to_datetime(row['最晚提交日期'], errors='coerce')
    date3 = pd.to_datetime(row['最早提交日期'], errors='coerce')
    
    # 如果 '最晚提交日期' 不存在，则返回四个 None
    if pd.isna(date2):
        return None, None, None, None
    
    # 如果 '发病日期' 存在
    if not pd.isna(date1):
        date_diff1 = (date2 - date1).days
        # 检查 date_diff1 是否为零
        if date_diff1 == 0:
            score_ratio1 = None
        else:
            score_ratio1 = (row['最晚分数'] - 48) / (date_diff1 / 30)
    else:
        date_diff1 = None
        score_ratio1 = None
    
    # 如果 '最早提交日期' 存在
    if not pd.isna(date3):
        date_diff2 = (date2 - date3).days
        # 检查 date_diff2 是否为零
        if date_diff2 == 0:
            score_ratio2 = None
        else:
            score_ratio2 = (row['最晚分数'] - row['最早分数']) / (date_diff2 / 30)
    else:
        date_diff2 = None
        score_ratio2 = None

    # 返回结果
    return date_diff1, date_diff2, score_ratio1, score_ratio2

In [47]:
df_long_date[['病程天数','提交天数','AR_K_all','AR_K_sub']] = df_long_date.apply(dur,axis=1,result_type='expand')    

In [3]:
df_long_date = df_long_date[df_long_date['病程天数']>=0]
df_long_date.to_csv('长病待筛选程表.csv',encoding='utf_8_sig')
# df_long_date = pd.read_csv('长病待筛选程表.csv')

In [ ]:
# 绘制成对关系图，只包含指定的变量
vars = ['病程天数','AR_K_Mon','最新分数']

g = sns.pairplot(df_long_date, vars=vars, palette='coolwarm', height=2.5)
# 调整图形尺寸
g.fig.set_size_inches(15, 15)
for ax in g.axes.flatten():
    ax.set_xticklabels([])
    ax.set_yticklabels([])
    # ax.tick_params(left=False, bottom=False)  # 移除刻度线
plt.xlim()
plt.savefig('D:/asl/fig.png', dpi=300)
plt.savefig('D:/asl/fig/lab_1_sta')
plt.show()


In [ ]:
plt.figure(figsize=(15, 6)) 
sns.scatterplot(df_long_date,x=df_long_date['病程天数'],y=df_long_date['AR_K_Mon'])
plt.savefig('D:/asl/fig.png', dpi=300)
plt.savefig('D:/asl/fig/lab_1_sta')
plt.show()